In [1]:
# PART 0: SETUP
# ==============================================================================
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [2]:
# ==============================================================================
# PART 1: DATA LOADING AND PREPROCESSING
# ==============================================================================
print("--- PART 1: Loading and Preprocessing Initial Data ---")

code_columns = {
    'ICD9_DGNS_CD_1': str, 'ICD9_DGNS_CD_2': str, 'ICD9_DGNS_CD_3': str,
    'ICD9_DGNS_CD_4': str, 'ICD9_DGNS_CD_5': str, 'ICD9_DGNS_CD_6': str,
    'ICD9_DGNS_CD_7': str, 'ICD9_DGNS_CD_8': str, 'ICD9_DGNS_CD_9': str,
    'ICD9_DGNS_CD_10': str, 'ADMTNG_ICD9_DGNS_CD': str, 'CLM_DRG_CD': str,
    'ICD9_PRCDR_CD_1': str, 'ICD9_PRCDR_CD_2': str, 'ICD9_PRCDR_CD_3': str,
    'ICD9_PRCDR_CD_4': str, 'ICD9_PRCDR_CD_5': str, 'ICD9_PRCDR_CD_6': str
}

beneficiary_2008 = pd.read_csv("D:/Jupyter/HealthArk_data/DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv")
beneficiary_2009 = pd.read_csv("D:/Jupyter/HealthArk_data/DE1_0_2009_Beneficiary_Summary_File_Sample_1.csv")
beneficiary_2010 = pd.read_csv("D:/Jupyter/HealthArk_data/DE1_0_2010_Beneficiary_Summary_File_Sample_1.csv")

chunk_size = 100000
    
inpatient_agg_list, inpatient_codes_list, inpatient_readmission_list = [], [], []
inpatient_iterator = pd.read_csv("D:/Jupyter/HealthArk_data/DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv", dtype=code_columns, chunksize=chunk_size)
for chunk in inpatient_iterator:
    inpatient_agg_list.append(chunk.groupby('DESYNPUF_ID').agg(Inpatient_Claim_Count=('CLM_ID', 'count'), Total_Inpatient_Payments=('CLM_PMT_AMT', 'sum')))
    inpatient_codes_list.append(chunk[['DESYNPUF_ID', 'ICD9_DGNS_CD_1']])
    chunk['CLM_ADMSN_DT'] = pd.to_datetime(chunk['CLM_ADMSN_DT'], format='%Y%m%d')
    chunk['CLM_THRU_DT'] = pd.to_datetime(chunk['CLM_THRU_DT'], format='%Y%m%d', errors='coerce')
    inpatient_readmission_list.append(chunk)

inpatient_agg = pd.concat(inpatient_agg_list).groupby(level=0).sum()
inpatient_codes = pd.concat(inpatient_codes_list)
inpatient_claims_raw = pd.concat(inpatient_readmission_list)
    
outpatient_agg_list, outpatient_codes_list = [], []
outpatient_iterator = pd.read_csv("D:/Jupyter/HealthArk_data/DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.csv", dtype=code_columns, engine='python', chunksize=chunk_size)
for chunk in outpatient_iterator:
    outpatient_agg_list.append(chunk.groupby('DESYNPUF_ID').agg(Outpatient_Claim_Count=('CLM_ID', 'count'), Total_Outpatient_Payments=('CLM_PMT_AMT', 'sum')))
    outpatient_codes_list.append(chunk[['DESYNPUF_ID', 'ICD9_DGNS_CD_1']])
        
outpatient_agg = pd.concat(outpatient_agg_list).groupby(level=0).sum()
outpatient_codes = pd.concat(outpatient_codes_list)

--- PART 1: Loading and Preprocessing Initial Data ---


In [3]:
# ==============================================================================
# PART 2: FEATURE ENGINEERING AND MERGING
# ==============================================================================
print("\n--- PART 2: Engineering Features and Merging Data ---")

all_beneficiaries = pd.concat([beneficiary_2008, beneficiary_2009, beneficiary_2010], ignore_index=True)
all_beneficiaries = all_beneficiaries.drop_duplicates(subset=['DESYNPUF_ID'], keep='last')
    
all_beneficiaries['BENE_BIRTH_DT'] = pd.to_datetime(all_beneficiaries['BENE_BIRTH_DT'], format='%m-%d-%Y')
all_beneficiaries['BENE_DEATH_DT'] = pd.to_datetime(all_beneficiaries['BENE_DEATH_DT'], format='%m-%d-%Y', errors='coerce')
reference_date = datetime(2010, 12, 31)
all_beneficiaries['Age'] = ((reference_date - all_beneficiaries['BENE_BIRTH_DT']).dt.days / 365.25).astype(int)
all_beneficiaries['Is_Dead'] = all_beneficiaries['BENE_DEATH_DT'].notna().astype(int)
chronic_condition_cols = [col for col in all_beneficiaries.columns if col.startswith('SP_')]
for col in chronic_condition_cols:
    all_beneficiaries[col] = all_beneficiaries[col].replace(2, 0)
all_beneficiaries['Chronic_Condition_Count'] = all_beneficiaries[chronic_condition_cols].sum(axis=1)
    
master_df = all_beneficiaries.merge(inpatient_agg, on='DESYNPUF_ID', how='left')
master_df = master_df.merge(outpatient_agg, on='DESYNPUF_ID', how='left')
claims_cols_to_fill = ['Inpatient_Claim_Count', 'Total_Inpatient_Payments', 'Outpatient_Claim_Count', 'Total_Outpatient_Payments']
master_df[claims_cols_to_fill] = master_df[claims_cols_to_fill].fillna(0)

inpatient_claims_raw = inpatient_claims_raw.sort_values(by=['DESYNPUF_ID', 'CLM_ADMSN_DT'])
inpatient_claims_raw['Next_Admission_Date'] = inpatient_claims_raw.groupby('DESYNPUF_ID')['CLM_ADMSN_DT'].shift(-1)
days_to_next_admission = (inpatient_claims_raw['Next_Admission_Date'] - inpatient_claims_raw['CLM_THRU_DT']).dt.days
inpatient_claims_raw['Was_Readmitted_in_30_Days'] = (days_to_next_admission <= 30).astype(int)
readmission_summary = inpatient_claims_raw.groupby('DESYNPUF_ID')['Was_Readmitted_in_30_Days'].max().reset_index()
readmission_summary = readmission_summary.rename(columns={'Was_Readmitted_in_30_Days': 'Had_30Day_Readmission_Ever'})
master_df_readmission = master_df.merge(readmission_summary, on='DESYNPUF_ID', how='left')
master_df_readmission['Had_30Day_Readmission_Ever'] = master_df_readmission['Had_30Day_Readmission_Ever'].fillna(0)
    
all_codes = pd.concat([inpatient_codes, outpatient_codes], ignore_index=True)
diagnosis_counts = all_codes.groupby('DESYNPUF_ID').size().reset_index(name='Total_Diagnosis_Count')
unique_diagnosis_counts = all_codes.groupby('DESYNPUF_ID')['ICD9_DGNS_CD_1'].nunique().reset_index(name='Unique_Diagnosis_Count')
master_df_enhanced = master_df_readmission.merge(diagnosis_counts, on='DESYNPUF_ID', how='left')
master_df_enhanced = master_df_enhanced.merge(unique_diagnosis_counts, on='DESYNPUF_ID', how='left')
master_df_enhanced[['Total_Diagnosis_Count', 'Unique_Diagnosis_Count']] = master_df_enhanced[['Total_Diagnosis_Count', 'Unique_Diagnosis_Count']].fillna(0)
categorical_cols = ['BENE_SEX_IDENT_CD', 'BENE_RACE_CD']
master_df_enhanced = pd.get_dummies(master_df_enhanced, columns=categorical_cols, drop_first=True)
    
try:
    drug_exposure = pd.read_excel("D:/Jupyter/HealthArk_data/drug_exposure.xlsx")
    person_mapping = pd.read_excel("D:/Jupyter/HealthArk_data/person.xlsx")
    person_id_map = person_mapping[['PERSON_ID', 'PERSON_SOURCE_VALUE']].rename(columns={'PERSON_SOURCE_VALUE': 'DESYNPUF_ID'})
    drug_exposure = drug_exposure.merge(person_id_map, on='PERSON_ID', how='left')
    
    if 'DESYNPUF_ID' in drug_exposure.columns:
        drug_counts = drug_exposure.groupby('DESYNPUF_ID').size().reset_index(name='Total_Drug_Count')
        unique_drug_counts = drug_exposure.groupby('DESYNPUF_ID')['DRUG_CONCEPT_ID'].nunique().reset_index(name='Unique_Drug_Count')
        avg_days_supply = drug_exposure.groupby('DESYNPUF_ID')['DAYS_SUPPLY'].mean().reset_index(name='Avg_Days_Supply')
        master_df_final = master_df_enhanced.merge(drug_counts, on='DESYNPUF_ID', how='left')
        master_df_final = master_df_final.merge(unique_drug_counts, on='DESYNPUF_ID', how='left')
        master_df_final = master_df_final.merge(avg_days_supply, on='DESYNPUF_ID', how='left')
        drug_feature_cols = ['Total_Drug_Count', 'Unique_Drug_Count', 'Avg_Days_Supply']
        master_df_final[drug_feature_cols] = master_df_final[drug_feature_cols].fillna(0)
except FileNotFoundError:
    print("Drug or Person files not found. Skipping drug features.")
    master_df_final = master_df_enhanced.copy()
    master_df_final[['Total_Drug_Count', 'Unique_Drug_Count', 'Avg_Days_Supply']] = 0


--- PART 2: Engineering Features and Merging Data ---


In [6]:
# ==============================================================================
# PART 3: ADVANCED FEATURE ENGINEERING (DIAGNOSIS-SPECIFIC FLAGS) - MEMORY SAFE
# ==============================================================================
print("\n--- PART 3: Engineering Diagnosis-Specific Features (Memory-Safe) ---")

# Define ICD-9 code prefixes for key conditions
diag_code_map = {
    'Has_Heart_Failure_Dx': ['428'],
    'Has_Kidney_Disease_Dx': ['585', '586'],
    'Has_Sepsis_Dx': ['038'],
    'Has_Pneumonia_Dx': ['480', '481', '482', '483', '485', '486'],
    'Has_COPD_Dx': ['491', '492', '496']
}

# Identify all diagnosis code columns
diag_cols = [f'ICD9_DGNS_CD_{i}' for i in range(1, 11)]

# List of claims files to process
claims_files = [
    "D:/Jupyter/HealthArk_data/DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv",
    "D:/Jupyter/HealthArk_data/DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.csv"
]

# Create a feature for each condition
for feature_name, codes in diag_code_map.items():
    
    all_patients_with_diagnosis = set()
    
    # Process each claims file in chunks
    for file in claims_files:
        print(f"Processing {file} for {feature_name}...")
        engine_type = 'python' if 'Outpatient' in file else 'c'
        iterator = pd.read_csv(file, dtype=code_columns, chunksize=100000, engine=engine_type)
        
        for chunk in iterator:
            # Check if any of the diagnosis columns contain any of the specified codes
            has_diagnosis = chunk[diag_cols].apply(
                lambda row: any(str(val).startswith(tuple(codes)) for val in row),
                axis=1
            )
            
            # Get the unique patient IDs from this chunk who have the diagnosis
            patients_in_chunk = chunk.loc[has_diagnosis, 'DESYNPUF_ID'].unique()
            
            # Add them to our master set of patients
            all_patients_with_diagnosis.update(patients_in_chunk)

    # Create the new feature in our final dataframe
    master_df_final[feature_name] = master_df_final['DESYNPUF_ID'].isin(all_patients_with_diagnosis).astype(int)
    print(f"Created feature: {feature_name} - Found in {len(all_patients_with_diagnosis)} patients.")


--- PART 3: Engineering Diagnosis-Specific Features (Memory-Safe) ---
Processing D:/Jupyter/HealthArk_data/DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv for Has_Heart_Failure_Dx...
Processing D:/Jupyter/HealthArk_data/DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.csv for Has_Heart_Failure_Dx...
Created feature: Has_Heart_Failure_Dx - Found in 25046 patients.
Processing D:/Jupyter/HealthArk_data/DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv for Has_Kidney_Disease_Dx...
Processing D:/Jupyter/HealthArk_data/DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.csv for Has_Kidney_Disease_Dx...
Created feature: Has_Kidney_Disease_Dx - Found in 19702 patients.
Processing D:/Jupyter/HealthArk_data/DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv for Has_Sepsis_Dx...
Processing D:/Jupyter/HealthArk_data/DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.csv for Has_Sepsis_Dx...
Created feature: Has_Sepsis_Dx - Found in 0 patients.
Processing D:/Jupyter/HealthArk_data/DE1_0_2008_to_2010_Inpatient_Cla

In [7]:
# ==============================================================================
# PART 4: FINAL MODEL TRAINING AND EVALUATION
# ==============================================================================
print("\n--- PART 4: Training and Evaluating the Ultimate Model ---")

y = master_df_final['Had_30Day_Readmission_Ever']
features_to_drop = ['DESYNPUF_ID', 'BENE_BIRTH_DT', 'BENE_DEATH_DT', 'Had_30Day_Readmission_Ever']
X = master_df_final.drop(columns=features_to_drop)
X = X.select_dtypes(include=['number'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

# Use the best parameters we found from our previous tuning
best_params = {
    'colsample_bytree': 0.8,
    'learning_rate': 0.05,
    'max_depth': 7,
    'n_estimators': 300,
    'subsample': 0.8
}

ultimate_model = XGBClassifier(
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1,
    **best_params
)

ultimate_model.fit(X_train, y_train)

y_pred = ultimate_model.predict(X_test)

print("\n--- Final Model Performance Evaluation (with Diagnosis-Specific Features) ---")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


--- PART 4: Training and Evaluating the Ultimate Model ---

--- Final Model Performance Evaluation (with Diagnosis-Specific Features) ---

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      0.94      0.97     22021
         1.0       0.47      0.96      0.63      1250

    accuracy                           0.94     23271
   macro avg       0.73      0.95      0.80     23271
weighted avg       0.97      0.94      0.95     23271

